# Get Embeddings from a Different Movie
### If you choose to analyze a different film, you can use this code to create your own embeddings json file.


#### You first need to generate thumbnails. I used this ffmpeg command:
~~~ 
ffmpeg -i movie-file.mkv -vf "fps=1,scale=320:-1" -q:v 2 thumbnails_folder/thumbnail_%04d.jpg
~~~ 

#### To pass the image files, you can run this command on your server from the thumbnails folder that serves the image files (so that CLIP can process them). 
~~~
python3 -m http.server 8811
~~~

#### Next, you should run the docker version of the replicate CLIP embedding on the same machine:
~~~
docker run -d -p 5000:5000 --gpus=all r8.im/andreasjansson/clip-features@sha256:75b33f253f7714a281ad3e9b28f63e3232d583716ef6718f2e46641077ea040a
~~~

#### With the webservice running on port 5000, then the code below should work.

In [12]:
all_json_responses = []

In [9]:
# %%timeit
import requests


# Function to download JSON response for a given thumbnail number
def download_json(thumbnail_number):
    url = "http://localhost:5000/predictions"
    headers = {"Content-Type": "application/json"}
    data = {
        "input": {
            "inputs": f"http://192.168.1.126:8811/thumbnail_{thumbnail_number:04d}.jpg"
        }
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to download JSON for thumbnail {thumbnail_number}. Status code: {response.status_code}")
        return None

# Function to iterate through a range of thumbnail numbers and collect JSON responses
def download_and_collect(start_thumbnail, end_thumbnail):
    all_json_responses = []
    for thumbnail_number in range(start_thumbnail, end_thumbnail + 1):
        json_response = download_json(thumbnail_number)
        if json_response is not None:
            json_response["output"][0]["input"] = json_response["output"][0]["input"].split("/")[-1]
            all_json_responses.append(json_response["output"][0])
#             break
    return all_json_responses

# Example usage
start_thumbnail = 1
end_thumbnail = 5441
all_json_responses = download_and_collect(start_thumbnail, end_thumbnail)

# Now `all_json_responses` contains a list of JSON responses for each thumbnail
# You can save it as a JSON file or process it further as needed


In [7]:
all_json_responses

[{'input': 'thumbnail_0001.jpg',
  'embedding': [-0.3141614496707916,
   0.21892297267913818,
   0.10630946606397629,
   -1.0737407207489014,
   -0.14155027270317078,
   0.10630404204130173,
   0.6218047738075256,
   -0.31840187311172485,
   0.13576644659042358,
   -0.8715311884880066,
   -0.028382882475852966,
   -0.7489148378372192,
   -0.4926939606666565,
   -0.8100481033325195,
   -0.03798965364694595,
   0.029798194766044617,
   -0.04123324155807495,
   -0.14592993259429932,
   -0.22605234384536743,
   0.06170624494552612,
   0.1367204487323761,
   0.5255715847015381,
   0.04877495393157005,
   -0.0702555924654007,
   -0.25562191009521484,
   0.24572815001010895,
   -0.13834156095981598,
   -0.452525794506073,
   0.426151305437088,
   -0.8569238185882568,
   -0.37028250098228455,
   0.033300697803497314,
   0.44591617584228516,
   0.1128811240196228,
   0.48469510674476624,
   -0.22459447383880615,
   0.12260260432958603,
   0.11869348585605621,
   -0.3089028596878052,
   -0.19949

In [10]:
len(all_json_responses)

5441

In [13]:
# all_json_responses[0].keys()

In [11]:
import json

# outfile = open("bee_movie_embeddings_1_second.json","w")
outfile = open("honey_i_shrunk_the_kids_movie_embeddings_1_second.json","w")
json.dump(all_json_responses,outfile,indent=2)
outfile.close()

In [23]:
more_file = json.load(open("bee_movie_embeddings_1_second.json"))

In [25]:
len(more_file)

5441